# Project 1 

## Group One: Covid 19 | Part One: Data Cleansing 

Team Members: Michael Bien, Lupita Lopez, Jinah Porter, Debra Wu

### Research Questions to Answer:

1. WORLD: Compare COVID deaths by GDP by Country: Does a higher GDP result in a lower number of COVID deaths?

2. US: Examine death rates by age group and determine is children under 10 die at a lower rate, adults 60+ at a higher rate

3. US: Compare cases/deaths by state

4. US: Does pre-COVID state GDP related to the cases/deaths in a given state

5. March shutdown, 4th of July and Memorial Day: compare 2 months of shutdown vs. reopening: how did reopening affect case and death rates? 

6. US: Is there a relationship between COVID cases/deaths and gender?

7. US: Is there a relationship between COVID cases/deaths and ethnicity?


In [205]:
#Import necessary libraries
import pandas as pd
import os
import requests
import numpy as np
import requests
from pprint import pprint
import json

Import World Bank data by Country; source: https://databank.worldbank.org/reports.aspx?source=2&series=NY.GDP.MKTP.CD&country=#

In [215]:
# Import world_bank_data.csv
file_path = os.path.join("..","source_data","world_bank_data.csv")
world_bank_df = pd.read_csv(file_path)
print(world_bank_df.shape)
world_bank_df.head()

(264, 3)


,Country Name,Country Code,2019 [YR2019]
0,Afghanistan,AFG,19101353833
1,Albania,ALB,15278077447
2,Algeria,DZA,1.69988E+11
3,American Samoa,ASM,..
4,Andorra,AND,3154057987


In [229]:
# Create list of countries
countries = world_bank_df["Country Name"]
#Create empty lists to hold api data
JH_deaths = []
JH_confirmed = []
JH_recovered = []

# John Hopkins base url
base_url = "https://covid19-stats-api.herokuapp.com/api/v1/cases?country="
for country in countries:
    request_url = base_url + country
    response = requests.get(request_url).json()
    try: 
        confirm_JH = response["confirmed"]
        JH_confirmed.append(confirm_JH)
        death_JH = response["deaths"]
        JH_deaths.append(death_JH)
        recovered_JH = response["recovered"]
        JH_recovered.append(death_JH)
        print(f"Processing country:{country}")
    except KeyError:
        print(f"<<<<< {country} has no data under the World Bank Name >>>>>")

Processing country:Afghanistan
Processing country:Albania
Processing country:Algeria
<<<<< American Samoa has no data under the World Bank Name >>>>>
Processing country:Andorra
Processing country:Angola
Processing country:Antigua and Barbuda
Processing country:Argentina
Processing country:Armenia
<<<<< Aruba has no data under the World Bank Name >>>>>
Processing country:Australia
Processing country:Austria
Processing country:Azerbaijan
<<<<< Bahamas, The has no data under the World Bank Name >>>>>
Processing country:Bahrain
Processing country:Bangladesh
Processing country:Barbados
Processing country:Belarus
Processing country:Belgium
Processing country:Belize
Processing country:Benin
<<<<< Bermuda has no data under the World Bank Name >>>>>
Processing country:Bhutan
Processing country:Bolivia
Processing country:Bosnia and Herzegovina
Processing country:Botswana
Processing country:Brazil
<<<<< British Virgin Islands has no data under the World Bank Name >>>>>
<<<<< Brunei Darussalam has

In [227]:
len(JH_confirmed)

162

In [189]:
# Import CDC data by country source: https://documenter.getpostman.com/view/922646/SzYbyci2?version=latest
#Create lists to gather data
country_region = []
deaths = []
recovered = []
confirmed = []
response = requests.get("https://2019ncov.asia/api/country_region").json()
for i in range(0,len(response["results"])):
    country = response["results"][i]["country_region"]
    country_region.append(country)
    death = response["results"][i]["deaths"]
    deaths.append(death)
    recover = response["results"][i]["recovered"]
    recovered.append(recover)
    confirm = response["results"][i]["confirmed"]
    confirmed.append(confirm)


In [190]:
# Create COVID_country dataframe,entry_df = pd.DataFrame[{"Country": country_region, "CDC Deaths": deaths, "CDC Recovered": recovered, "CDC ", "CDC Confrmed": confirmed}\
country_df = pd.DataFrame({"Country": country_region, "CDC Deaths": deaths, "CDC Recovered": recovered, "CDC Confirmed": confirmed})
country_df.head()

,Country,CDC Deaths,CDC Recovered,CDC Confirmed
0,Afghanistan,1425,32073,38772
1,Albania,338,6615,11520
2,Algeria,1620,34204,48496
3,Andorra,53,945,1438
4,Angola,136,1324,3439


In [202]:
# get Johns Hopkins Case Data for each country
# create empty lists to gather data
country_df["JH Confirmed"] = ""
country_df["JH Deaths"] = ""
country_df["JH Recovered"] = ""

base_url = "https://covid19-stats-api.herokuapp.com/api/v1/cases?"
# API call loop
for index,row in country_df.iterrows():
    try:
        request_url = base_url + row["Country"]
        response = requests.get(request_url).json()
        confirm_JH = response["confirmed"]
        row["JH Confirmed"] = confirm_JH
        death_JH = response["deaths"]
        row["JH Deaths"] = death_JH
        recover_JH = response["recovered"]
        row["JH Recovered"] = recover_JH
        pass
    except KeyError:
        print(f"{row[country]} errored out")



In [203]:
print(country_df).head()

            Country  CDC Deaths  CDC Recovered  CDC Confirmed JH Confirmed  \
0       Afghanistan        1425          32073          38772                
1           Albania         338           6615          11520                
2           Algeria        1620          34204          48496                
3           Andorra          53            945           1438                
4            Angola         136           1324           3439                
..              ...         ...            ...            ...          ...   
210         Vietnam          35            926           1063                
211  Western Sahara           1              8             10                
212           Yemen         583           1215           2013                
213          Zambia         320          12380          13720                
214        Zimbabwe         224           5690           7531                

    JH Deaths JH Recovered  
0                           
1    

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:

response = requests.get("https://covid19-stats-api.herokuapp.com/api/v1/cases?country=India").json()
blah = (response["confirmed"])
print(response)

In [ ]:
country_df.head()